In [1]:
from math import inf as infinity
from random import choice
import platform
import time
from os import system

"""
An implementation of Minimax AI Algorithm in Tic Tac Toe, using Python.
This software is available under GPL license.
Author: Clederson Cruz
Year: 2017
License: GNU GENERAL PUBLIC LICENSE (GPL)
"""

HUMANO = -1
COMPUTADOR = +1
board = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
movimientosH = []
movimientosC = []
contadorH = 0
contadorC = 0
def evaluate(estado):
    """
    Función de evaluación del estado de finalización del juego.
     : parametro: estado, estado actual del tablero
     : devuelve: +1 si COMPUTADOR gana; -1 si el HUMANO gana; 0 en caso de empate
    """
    if wins(estado, COMPUTADOR):
        score = +1
    elif wins(estado, HUMANO):
        score = -1
    else:
        score = 0

    return score
#----------------------PRIMER PARCIAL-----------------------------------------
# PARA LA VALIDACION DE QUE ESTAN SEGUIDOS UNA A OTRO LA CONDICION SIEMPRE [1,0] o [0,1] ESO INDICA QUE ES MOVIENTO VERTICA O HORIZONTAL

def correlacion(lista):
    # Ordenar por coordenada x y luego por coordenada y
    lista_ordenada = sorted(lista, key=lambda punto: (punto[0], punto[1]))
    contador = 0
    for i in range(len(lista_ordenada)-1):
      valor1 = lista_ordenada[i]
      for j in range(i+1, len(lista_ordenada)):
          valor2 = lista_ordenada[j]
          diferencia = [valor2[0]-valor1[0], valor2[1]-valor1[1]]
          if(diferencia == [0,1] or diferencia == [1,0]):
              contador += 1
          diferencia = []
    return contador

def wins(estado, player):
    """
    Esta funcion verifica si un jugador especifico gana. Posibilidades:
    * Tres filas    [X X X] or [O O O]
    * Tres columnas    [X X X] or [O O O]
    * Dos diagonales [X X X] or [O O O]
    :parametro estado, el estado del tablero actual
    :parametro player: un HUMANOo o un COMPUTADORutador
    :devuelve: True si un jugador gana
    """

    win_state = [
        #HORIZONTALES
        [estado[0][0], estado[0][1], estado[0][2]],
        [estado[1][0], estado[1][1], estado[1][2]],
        [estado[2][0], estado[2][1], estado[2][2]],
        # #VERTICALES
        [estado[0][0], estado[1][0], estado[2][0]],
        [estado[0][1], estado[1][1], estado[2][1]],
        [estado[0][2], estado[1][2], estado[2][2]]
        # #DIAGONALES
        # [estado[0][0], estado[1][1], estado[2][2]],
        # [estado[2][0], estado[1][1], estado[0][2]],
    ]

    if [player, player, player] in win_state:
      return False
    # else:
    #     return False
def game_over(estado):
    """
    Esa funcion verifica si el HUMANO o el COMPUTADOR gana
    :parametro estado, estado del tablero actual
    :devuelve: True si el HUMANO o el COMPUTADOR gana
    """
    return wins(estado, HUMANO) or wins(estado, COMPUTADOR)


def empty_cells(estado):
    """
    Cada celda vacía se agregará a la lista de celdas
    :parametro estado, estado de tablero actual
    :devuelve, una lista de las celdas vacias
    """
    cells = []

    for x, fila in enumerate(estado):
        for y, cell in enumerate(fila):
            if cell == 0:
                cells.append([x, y])
    return cells

def valid_move(x, y):
    """
    Un movimiento es válido si la celda elegida está vacía
    :parametro x, coordenada X
    :parametro y, coordenada Y 
    :devuelve: True si la posicion del tablero[x][y] esta vacia
    """
    if [x, y] in empty_cells(board):
        return True
    else:
        return False

def set_move(x, y, player):
    """
    Establece un movimiento en el tablero, si las coordenadas son validas
    :parametro x, coordenada X
    :parametro y, coordenada Y
    :parametro player, jugador actual
    """
    if valid_move(x, y):
        board[x][y] = player
        #----------------------PRIMER PARCIAL-----------------------------------------
        #CAPTURAMOS LAS COORDENADAS DE LOS MOVIMENTOS FILA = X y COLUMNA = Y , DE LOS JUGADORES
        #Y AGREGAMOS A UN ARREGLO
        if player == HUMANO:
            movimientosH.append([x, y])
        if player == COMPUTADOR:
            movimientosC.append([x, y])
        return True
    else:
        return False


def minimax(estado, depth, player):
    """
    Funcion IA que elige la mejor movida
    AI function that choice the best move
    :parametro estado, estado actual en el tablero
    :param depth, indice del nodo en el arbol (0 <= depth < 9), pero nunca nueve.
    :param player, un HUMANO o un COMPUTADOR
    :devuelve, una lista con [la mejor fila, la mejor columna, el mejor score]
    """
    if player == COMPUTADOR:
        best = [-1, -1, -infinity]
    else:
        best = [-1, -1, +infinity]

    if depth == 0 or game_over(estado):
        score = evaluate(estado)
        return [-1, -1, score]

    for cell in empty_cells(estado):
        x, y = cell[0], cell[1]
        estado[x][y] = player
        score = minimax(estado, depth - 1, -player)
        estado[x][y] = 0
        score[0], score[1] = x, y

        #Esto nos dice quien quieres que gane en este caso el COMPUTADOR
        if player == COMPUTADOR:
            if score[2] > best[2]:
                best = score  # max value
        else:
            if score[2] < best[2]:
                best = score  # min value

    return best


def clean():
    """
    Clears the console
    """
    os_name = platform.system().lower()
    if 'windows' in os_name:
        system('cls')
    else:
        system('clear')


def render(estado, c_choice, h_choice):
    """
    Print the board on console
    :param estado: current estado of the board
    """

    chars = {
        -1: h_choice,
        +1: c_choice,
        0: ' '
    }
    str_line = '---------------'

    print('\n' + str_line)
    for fila in estado:
        for cell in fila:
            symbol = chars[cell]
            print(f'| {symbol} |', end = '')
        print('\n' + str_line)


def ai_turn(c_choice, h_choice):
    """
    Esta funcion llama a la funcion minimax si la profundidad es < 9,
    caso contrario esta elige una coordenada aleatoria.
    :param c_choice: COMPUTADOR elije X o O
    :param h_choice: HUMANO elije X o O
    :return:
    """
    depth = len(empty_cells(board))
    #----------------------PRIMER PARCIAL-----------------------------------------
    # PARA QUE QUEDE UNA CELDA VACIA PONEMOS 1 A CAMBIO DE 0
    if depth == 1 or game_over(board):
        return

    clean()
    print(f'Juega COMPUTADOR [{c_choice}]')
    render(board, c_choice, h_choice)

    if depth == 9:
        x = choice([0, 1, 2])
        y = choice([0, 1, 2])
    else:
        move = minimax(board, depth, COMPUTADOR)
        x, y = move[0], move[1]

    set_move(x, y, COMPUTADOR)
    time.sleep(1)

def HUMANO_turn(c_choice, h_choice):
    """
    El HUMANO juega eligiendo una movida valida.
    :param c_choice: COMPUTADORuter's choice X or O
    :param h_choice: HUMANO's choice X or O
    :return:
    """
    depth = len(empty_cells(board))
    if depth == 0 or game_over(board):
        return

    # Dictionary of valid moves
    move = -1
    moves = {
        1: [0, 0], 2: [0, 1], 3: [0, 2],
        4: [1, 0], 5: [1, 1], 6: [1, 2],
        7: [2, 0], 8: [2, 1], 9: [2, 2],
    }

    clean()
    print(f'turno HUMANO [{h_choice}]')
    render(board, c_choice, h_choice)

    while move < 1 or move > 9:
        try:
            move = int(input('Use los numeros (1..9): '))
            coord = moves[move]
            can_move = set_move(coord[0], coord[1], HUMANO)

            if not can_move:
                print('Bad move')
                move = -1
        except (EOFError, KeyboardInterrupt):
            print('Bye')
            exit()
        except (KeyError, ValueError):
            print('Bad choice')


def main():
    """
    Main function that calls all functions
    """
    clean()
    h_choice = ''  # X or O
    c_choice = ''  # X or O
    first = ''  # if HUMANO is the first

    # HUMANO chooses X or O to play
    while h_choice != 'O' and h_choice != 'X':
        try:
            print('')
            h_choice = input('Choose X or O\nChosen: ').upper()
        except (EOFError, KeyboardInterrupt):
            print('Bye')
            exit()
        except (KeyError, ValueError):
            print('Bad choice')

    # Setting COMPUTADORuter's choice
    if h_choice == 'X':
        c_choice = 'O'
    else:
        c_choice = 'X'

    # HUMANO may starts first
    clean()
    while first != 'Y' and first != 'N':
        try:
            first = input('First to start?[y/n]: ').upper()
        except (EOFError, KeyboardInterrupt):
            print('Bye')
            exit()
        except (KeyError, ValueError):
            print('Bad choice')

    # Main loop of this game
    #----------------------PRIMER PARCIAL-----------------------------------------
    # PARA QUE QUEDE UNA CELDA VACIA PONEMOS 1 A CAMBIO DE 0
    while len(empty_cells(board)) > 1 and not game_over(board):
        if first == 'N':
            ai_turn(c_choice, h_choice)
            first = ''

        HUMANO_turn(c_choice, h_choice)
        ai_turn(c_choice, h_choice)

    # Game over message
    if wins(board, HUMANO):
        clean()
        print(f'HUMANO turn [{h_choice}]')
        render(board, c_choice, h_choice)
        # print('YOU WIN!')
    elif wins(board, COMPUTADOR):
        clean()
        print(f'COMPUTADORuter turn [{c_choice}]')
        render(board, c_choice, h_choice)
        # print('YOU LOSE!')
    else:
        clean()
        render(board, c_choice, h_choice)
        # print('DRAW!')
    #----------------------PRIMER PARCIAL-----------------------------------------
    # LLAMAMOS A NUESTRA FUNCION CORRELACION O RELACION ENTRE CELDAS PARA Y LAS CONTAMOS
    contadorH = correlacion(movimientosH)
    contadorC = correlacion(movimientosC)
    if contadorH > contadorC:
      print("JUGADOR: ",contadorH)
      print("COMPUTADOR: ",contadorC)
      print("---------GANASTE---------")
    elif contadorH == contadorC:
      print("JUGADOR: ",contadorH)
      print("COMPUTADOR: ",contadorC)
      print("---------EMPATE---------")
    else:
      print("JUGADOR: ",contadorH)
      print("COMPUTADOR: ",contadorC)
      print("---------PERDISTE---------")

    exit()


if __name__ == '__main__':
    main()


Choose X or O
Chosen: x
First to start?[y/n]: y
turno HUMANO [X]

---------------
|   ||   ||   |
---------------
|   ||   ||   |
---------------
|   ||   ||   |
---------------
Use los numeros (1..9): 9
Juega COMPUTADOR [O]

---------------
|   ||   ||   |
---------------
|   ||   ||   |
---------------
|   ||   || X |
---------------
turno HUMANO [X]

---------------
| O ||   ||   |
---------------
|   ||   ||   |
---------------
|   ||   || X |
---------------
Use los numeros (1..9): 8
Juega COMPUTADOR [O]

---------------
| O ||   ||   |
---------------
|   ||   ||   |
---------------
|   || X || X |
---------------
turno HUMANO [X]

---------------
| O || O ||   |
---------------
|   ||   ||   |
---------------
|   || X || X |
---------------
Use los numeros (1..9): 5
Juega COMPUTADOR [O]

---------------
| O || O ||   |
---------------
|   || X ||   |
---------------
|   || X || X |
---------------
turno HUMANO [X]

---------------
| O || O || O |
---------------
|   || X ||   |

In [7]:
listaH = [[0, 1], [0, 2], [0, 1], [1, 1]]
# print(listaH[0])
contador = 0
for i in range(len(listaH)):
    listaH[i] = [fila,columna]
    print(listaH[i])
    # if listaH[i] == listaH[i+1][::-1]:
    #     contador += 1

# print(contador)

NameError: ignored

In [2]:
listaH = [[0,1], [0,2], [1,1]]

lista1 = [0, 1]
lista2 = [0, 2]
contador = 0
resultado = []
for i in range(len(lista1)):
    resultado.append(lista1[i] - lista2[i])
    if(resultado == [0,1] or resultado == [1,0]):
      contador += 1
    resultado = []
print(contador)
print(resultado)

0
[0, -1]


In [6]:
listaH = [[0,0], [0,1], [1,1], [2,1]]
contador = 0
resultado = []
for lista in listaH:
    for i in range(len(lista)):
        resultado.append(lista[i-1] - lista[i])
    if(resultado == [0,1] or resultado == [1,0]):
        contador += 1
    resultado = []
print(contador)
print(resultado)

0
[]


In [14]:
resultado = []
contador = 0
listaH = [[0,0], [0,1], [1,1], [2,1]]
for i in range(len(listaH)-1):
    valor1 = listaH[i]
    valor2 = listaH[i+1]
    resultado = valor2 - valor1
    

TypeError: ignored

In [18]:
contador = 0
listaH = [[0,0], [0,1], [1,1], [2,1]]
for i in range(len(listaH)-1):
    valor1 = listaH[i]
    valor2 = listaH[i+1]
    diferencia = [valor2[0]-valor1[0], valor2[1]-valor1[1]]
    if(diferencia == [0,1] or diferencia == [1,0]):
        contador += 1
    diferencia = []
print(contador)

3


In [5]:
vector = [[0, 0], [2, 2], [1, 0], [1, 2]]

# Ordenar por coordenada x y luego por coordenada y
vector = sorted(vector, key=lambda punto: (punto[0], punto[1]))

print(vector)

[[0, 0], [1, 0], [1, 2], [2, 2]]
